In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import joblib

In [3]:
import pandas as pd
from utils.preprocess import treat_dataset_pandas_init, build_preprocessing_pipeline,INIT_NUMERICAL_COLS, numerical_features

from utils.metrics import rmsle_metric

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.metrics import root_mean_squared_log_error
from sklearn.ensemble import StackingRegressor, HistGradientBoostingRegressor
import numpy as np
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from utils.categoricals import CategoricalToStringTransformer, CategoricalEncoder, CatogoricalUnknownTransformer

from sklearn.metrics import mean_squared_log_error


In [4]:
train_data = pd.read_csv("playground-series-s4e12/train.csv")#.sample(frac=0.1)
test_data = pd.read_csv("playground-series-s4e12/test.csv")

treated_dataset = treat_dataset_pandas_init(train_data, process_as_category=True)
treated_dataset_test = treat_dataset_pandas_init(test_data, process_as_category=True)

non_loged_train_local, non_loged_test_local = joblib.load("cat_non_loged_local.pkl")
non_loged_train, non_loged_test = joblib.load("cat_non_loged.pkl")

treated_dataset['non_log_oof_prediction'] = non_loged_train
treated_dataset['non_log_oof_prediction_local'] = non_loged_train_local
treated_dataset_test['non_log_oof_prediction'] = non_loged_test
treated_dataset_test['non_log_oof_prediction_local'] = non_loged_test_local

X_train = treated_dataset.drop(columns=["Premium Amount"])
y_train = treated_dataset["Premium Amount"]


In [5]:

# Define a 60-40 split
#splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
splitter = KFold(n_splits=5, shuffle=True, random_state=42)


In [6]:
xgboost_params = {'learning_rate': 0.04656560304624032, 'max_depth': 10, 'min_child_weight': 50, 'subsample': 0.9730484625285342, 'colsample_bytree': 0.5743561244230219, 'gamma': 7.9578377008338235, 'lambda': 9.268384283962487, 'alpha': 9.9663808717552, 'n_estimators': 754}

In [7]:
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)
    
    full_pipeline = Pipeline([
        #("categorical_encoder", CategoricalEncoder()),
        ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist"))
    ])
    full_pipeline.fit(X_train_fold, y_train_fold)
    
    train_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_val_fold)), np.expm1(y_val_fold))

    print(f"Train: {train_score}, Test: {val_score}")

Starting


d:\Development\insurance\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [15:18:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Train: 1.0236011483923748, Test: 1.0314171277936783
Starting
Train: 1.0236405075379336, Test: 1.0308962014307577
Starting
Train: 1.0233155884810634, Test: 1.0327360956412754
Starting
Train: 1.024433375779329, Test: 1.0296840045463045
Starting
Train: 1.0234516441967898, Test: 1.0319384861214653


In [9]:
xgb_model = xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist")
xgb_model.fit(X_train, np.log1p(y_train))

test_predictions = np.expm1(xgb_model.predict(treated_dataset_test))
test_predictions = np.clip(test_predictions, 20, 4000)


In [11]:
submission = pd.DataFrame({"id": test_data["id"], "Premium Amount": test_predictions})
submission.to_csv("submission_xgb_local.csv", index=False)
